# Long-term Chat Memory Agent with LangGraph
---

This tutorial walks you through how to build an agent with long-term memory capabilities. 
With this setup, the agent can store, recall, and leverage memories to create richer, more personalized interactions with users.

In this tutorial, "memory" takes two main forms:

- Text-based insights generated by the memory agent
- Structured knowledge about entities, stored as (subject, predicate, object) triples

These memories can later be retrieved or queried semantically, allowing the agent to provide user-specific context across conversations.
The key idea here is that by saving memories, the agent can retain information about users that persists across multiple conversation threads.

- Reference: https://python.langchain.com/docs/versions/migrating_memory/long_term_memory_agent/

In [ ]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv()

aoai_api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
aoai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
aoai_emb_deployment_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")

if not aoai_api_version:
    aoai_api_version = os.getenv("OPENAI_API_VERSION")

try:
    print("=== Initialized AzuureOpenAI client ===")
    print(f"AZURE_OPENAI_ENDPOINT={aoai_api_endpoint}")
    print(f"AZURE_OPENAI_API_VERSION={aoai_api_version}")
    print(f"AZURE_OPENAI_DEPLOYMENT_NAME={aoai_deployment_name}")
    print(f"AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME={aoai_emb_deployment_name}")
except (ValueError, TypeError) as e:
    print(e)

<br>

## 🧪 1. Preparation and Define the Agentic Architecture
---
### PDF retrieval

In [ ]:
from azure_genai_utils.rag.pdf import PDFRetrievalChain

pdf_path = "../../sample-docs/AutoGen-paper.pdf"

pdf = PDFRetrievalChain(
    source_uri=[pdf_path],
    loader_type="PDFPlumber",
    model_name=aoai_deployment_name,
    embedding_name=aoai_emb_deployment_name,
    chunk_size=500,
    chunk_overlap=50,
).create_chain()

pdf_retriever = pdf.retriever
pdf_chain = pdf.chain

### Define vector store 

In [ ]:
import json
from typing import List, Literal, Optional

import tiktoken
from langchain_core.documents import Document
from langchain_core.messages import get_buffer_string
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import AzureChatOpenAI
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode

embeddings = AzureOpenAIEmbeddings(
    model=aoai_emb_deployment_name,
    chunk_size=1000,
)
recall_vector_store = InMemoryVectorStore(embeddings)

### Define tools

In [ ]:
import uuid


def get_user_id(config: RunnableConfig) -> str:
    user_id = config["configurable"].get("user_id")
    if user_id is None:
        raise ValueError("User ID needs to be provided to save a memory.")

    return user_id


@tool
def save_recall_memory(memory: str, config: RunnableConfig) -> str:
    """Save memory to vectorstore for later semantic retrieval."""
    user_id = get_user_id(config)
    document = Document(
        page_content=memory, id=str(uuid.uuid4()), metadata={"user_id": user_id}
    )
    recall_vector_store.add_documents([document])
    return memory


@tool
def search_recall_memories(query: str, config: RunnableConfig) -> List[str]:
    """Search for relevant memories."""
    user_id = get_user_id(config)

    def _filter_function(doc: Document) -> bool:
        return doc.metadata.get("user_id") == user_id

    documents = recall_vector_store.similarity_search(
        query, k=3, filter=_filter_function
    )
    return [document.page_content for document in documents]


@tool
def pdf_retrieve(query: str, config: RunnableConfig):
    """Retrieve information regarding AutoGen paper. If the query asks for details about AutoGen, use this tool."""
    print("\n==== [RETRIEVE] ====\n")
    documents = pdf_retriever.invoke(query)
    return [document.page_content for document in documents]

### Define Web search tool

In [ ]:
from azure_genai_utils.tools import BingSearch

WEB_SEARCH_FORMAT_OUTPUT = False

web_search_tool = BingSearch(
    max_results=1,
    locale="en-US",
    include_news=False,
    include_entity=False,
    format_output=WEB_SEARCH_FORMAT_OUTPUT,
)

# Define the tools to be used in the state graph
tools = [save_recall_memory, search_recall_memories, pdf_retrieve, web_search_tool]

### Define the prompt template for the agent

In [ ]:
system_prompt = """
You are a helpful assistant with advanced long-term memory capabilities. 
Powered by a stateless LLM, you must rely on external memory to store information between conversations. 
Utilize the available memory tools to store and retrieve important details that will help you better attend to the user's needs and understand their context.

## Memory Usage Guidelines:
1. Actively use memory tools (save_core_memory, save_recall_memory) to build a comprehensive understanding of the user.
2. Make informed suppositions and extrapolations based on stored memories.
3. Regularly reflect on past interactions to identify patterns and preferences.
4. Update your mental model of the user with each new piece of information.
5. Cross-reference new information with existing memories for consistency.
6. Prioritize storing emotional context and personal values alongside facts.
7. Use memory to anticipate needs and tailor responses to the user's style.
8. Recognize and acknowledge changes in the user's situation or perspectives over time.
9. Leverage memories to provide personalized examples and analogies.
10. Recall past challenges or successes to inform current problem-solving.

## Constraint
1. Review the provided context thoroughly and extract key details related to the question.
2. Craft a precise answer based on the relevant information.
3. Keep the answer concise but logical/natural/in-depth.
4. If the retrieved context does not contain relevant information or no context is available, respond with: 'I can't find the answer to that question in the context.'

## Recall Memories
Recall memories are contextually retrieved based on the current conversation:
{recall_memories}

## Instructions
Engage with the user naturally, as a trusted colleague or friend. There's no need to explicitly mention your memory capabilities. 
Instead, seamlessly incorporate your understanding of the user into your responses. 
Be attentive to subtle cues and underlying emotions. Adapt your communication style to match the user's preferences and current emotional state. 
Use tools to persist information you want to retain in the next conversation. 
If you do call tools, all text preceding the tool call is an internal message. 
Respond AFTER calling the tool, once you have confirmation that the tool completed successfully.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("placeholder", "{messages}"),
    ]
)

### Define state, nodes and edges

- agent: Process the current state (state contains previous messages and memory) and generate a response. 
- load_memories: Load memories for the current conversation.
- route_tools: Determine whether to use tools or end the conversation based on the last message.

In [ ]:
model = AzureChatOpenAI(model_name=aoai_deployment_name)
model_with_tools = model.bind_tools(tools)
tokenizer = tiktoken.encoding_for_model(aoai_deployment_name)

from typing import List
from typing_extensions import TypedDict, Annotated


class State(MessagesState):
    # add memories that will be retrieved based on the conversation context
    recall_memories: Annotated[List[str], "List of recall memories"]


def agent(state: State) -> State:
    """Process the current state and generate a response using the LLM.

    Args:
        state (schemas.State): The current state of the conversation.

    Returns:
        schemas.State: The updated state with the agent's response.
    """
    bound = prompt | model_with_tools
    recall_str = (
        "<recall_memory>\n" + "\n".join(state["recall_memories"]) + "\n</recall_memory>"
    )
    prediction = bound.invoke(
        {
            "messages": state["messages"],
            # "context": format_docs(state["documents"]),
            "recall_memories": recall_str,
        }
    )
    return {
        "messages": [prediction],
    }


def load_memories(state: State, config: RunnableConfig) -> State:
    """Load memories for the current conversation.

    Args:
        state (schemas.State): The current state of the conversation.
        config (RunnableConfig): The runtime configuration for the agent.

    Returns:
        State: The updated state with loaded memories.
    """
    convo_str = get_buffer_string(state["messages"])
    convo_str = tokenizer.decode(tokenizer.encode(convo_str)[:2048])
    recall_memories = search_recall_memories.invoke(convo_str, config)
    return {
        "recall_memories": recall_memories,
    }


def route_tools(state: State):
    """Determine whether to use tools or end the conversation based on the last message.

    Args:
        state (schemas.State): The current state of the conversation.

    Returns:
        Literal["tools", "__end__"]: The next step in the graph.
    """
    msg = state["messages"][-1]
    if msg.tool_calls:
        return "tools"

    return END

### Construct the state graph

In [ ]:
# Create the graph and add nodes
builder = StateGraph(State)

builder.add_node(load_memories)
builder.add_node(agent)
builder.add_node("tools", ToolNode(tools))

# Add edges to the graph
builder.add_edge(START, "load_memories")
builder.add_edge("load_memories", "agent")
builder.add_conditional_edges("agent", route_tools, ["tools", END])
builder.add_edge("tools", "agent")

# Compile the graph
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [ ]:
from azure_genai_utils.graphs import visualize_langgraph

visualize_langgraph(graph, xray=True)

<br>

## 🧪 2. Run the agent
---

In [ ]:
def pretty_print_stream_chunk(chunk):
    for node, updates in chunk.items():
        print(f"🔄 Update from Node: \033[1;36m{node}\033[0m 🔄")
        if "messages" in updates:
            updates["messages"][-1].pretty_print()
        else:
            print(updates)

        print("\n")

### Person 1. Daekeun

This is a scenario for user Daekeun. Daekeun mentions his role and interests when starting a conversation, and these are stored in memory. In subsequent questions, the agent extracts the most relevant context to the memory as top_k and conducts a conversation based on that context. The memory is continuously managed for the same user, and only similar memories are extracted as top_k, so the context is not forgotten even if the conversation is long.

In [ ]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    recursion_limit=10, configurable={"user_id": "1", "thread_id": "1"}
)

for chunk in graph.stream(
    {
        "messages": [
            (
                "user",
                "Daekeun is a Machine Learning geek. He loves to learn AIML new things.",
            )
        ]
    },
    config=config,
):
    pretty_print_stream_chunk(chunk)

Note that the response. You can see recall_memories are updated.

In [ ]:
for chunk in graph.stream(
    {"messages": [("user", "Daekeun provides AIML technology support at Microsoft.")]},
    config=config,
):
    pretty_print_stream_chunk(chunk)

This code cell shows the agent tries to retrieve the information from the PDF document.

In [ ]:
for chunk in graph.stream(
    {"messages": [("user", "Daekeun wants to know AutoGen")]}, config=config
):
    pretty_print_stream_chunk(chunk)

In [ ]:
for chunk in graph.stream(
    {"messages": [("user", "What is AutoGen's main featrues?")]}, config=config
):
    pretty_print_stream_chunk(chunk)

The agent tries to find materials from the web with the saved memories.

In [ ]:
for chunk in graph.stream(
    {
        "messages": [
            (
                "user",
                "Daekeun wants to study AutoGen in 2 weeks. Please recommend Microsoft's website or appropriate learning material.",
            )
        ]
    },
    config=config,
):
    pretty_print_stream_chunk(chunk)

### Person 2. Hyo

Switch the topic to another User Hyo. ​​Since memory is managed per user, please make sure that the memory is empty for the new user.

In [ ]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    recursion_limit=10, configurable={"user_id": "2", "thread_id": "1"}
)


for chunk in graph.stream(
    {"messages": [("user", "Hyo is a big fan of Microsoft")]}, config=config
):
    pretty_print_stream_chunk(chunk)

In [ ]:
for chunk in graph.stream(
    {"messages": [("user", "Hyo is interested in AutoGen and Semantic Kernel")]},
    config=config,
):
    pretty_print_stream_chunk(chunk)

In [ ]:
for chunk in graph.stream(
    {"messages": [("user", "Where is learning materials?")]}, config=config
):
    pretty_print_stream_chunk(chunk)

In [ ]:
for chunk in graph.stream(
    {"messages": [("user", "what's the address for joe's in greenwich village?")]},
    config=config,
):
    pretty_print_stream_chunk(chunk)

<br>

## 🧪 3. Adding structured memories
---

We have represented memory as a string so far. This is the baseline for storing memory in vector storage and is simple to implement. However, if you find it useful to use a different persistence backend, such as a graph database, you can update your application to create a memory.

In [ ]:
recall_vector_store = InMemoryVectorStore(embeddings)

In [ ]:
from typing_extensions import TypedDict


class KnowledgeTriple(TypedDict):
    subject: str
    predicate: str
    object_: str


### Baseline implementation of save_recall_memory (Represented memory as a string)
# @tool
# def save_recall_memory(memory: str, config: RunnableConfig) -> str:
#     """Save memory to vectorstore for later semantic retrieval."""
#     user_id = get_user_id(config)
#     document = Document(
#         page_content=memory, id=str(uuid.uuid4()), metadata={"user_id": user_id}
#     )
#     recall_vector_store.add_documents([document])
#     return memory


@tool
def save_recall_memory(memories: List[KnowledgeTriple], config: RunnableConfig) -> str:
    """Save memory to vectorstore for later semantic retrieval."""
    user_id = get_user_id(config)
    for memory in memories:
        serialized = " ".join(memory.values())
        document = Document(
            serialized,
            id=str(uuid.uuid4()),
            metadata={
                "user_id": user_id,
                **memory,
            },
        )
        recall_vector_store.add_documents([document])
    return memories

In [ ]:
tools = [save_recall_memory, search_recall_memories, pdf_retrieve, web_search_tool]
model_with_tools = model.bind_tools(tools)

# Create the graph and add nodes
builder = StateGraph(State)
builder.add_node(load_memories)
builder.add_node(agent)
builder.add_node("tools", ToolNode(tools))

# Add edges to the graph
builder.add_edge(START, "load_memories")
builder.add_edge("load_memories", "agent")
builder.add_conditional_edges("agent", route_tools, ["tools", END])
builder.add_edge("tools", "agent")

# Compile the graph
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

### Person 3. Wonchan


In [ ]:
config = {"configurable": {"user_id": "3", "thread_id": "1"}}

for chunk in graph.stream({"messages": [("user", "Hi I am Wonchan.")]}, config=config):
    pretty_print_stream_chunk(chunk)

In [ ]:
for chunk in graph.stream(
    {
        "messages": [
            (
                "user",
                "I am non-tech, but interested in Microsoft's multi-agent strategy and tech stack like AutoGen.",
            )
        ]
    },
    config=config,
):
    pretty_print_stream_chunk(chunk)

The memories generated from one thread are accessed in another thread from the same user:

In [ ]:
config = {"configurable": {"user_id": "3", "thread_id": "2"}}

for chunk in graph.stream(
    {
        "messages": [
            ("user", "Recommend me a website where I can easily try AutoGen hands-on")
        ]
    },
    config=config,
):
    pretty_print_stream_chunk(chunk)

In [ ]:
for chunk in graph.stream(
    {
        "messages": [
            (
                "user",
                "Recommend other multi-agent frameworks to me inorder to learn about other companies' multi-agent strategies",
            )
        ]
    },
    config=config,
):
    pretty_print_stream_chunk(chunk)

In [ ]:
records = recall_vector_store.similarity_search(
    "multi-agent", k=3, filter=lambda doc: doc.metadata["user_id"] == "3"
)
print(records)

Optionally, for illustrative purposes we can visualize the knowledge graph extracted by the model:

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

# Fetch records
records = recall_vector_store.similarity_search(
    "multi-agent", k=2, filter=lambda doc: doc.metadata["user_id"] == "3"
)


# Plot graph
plt.figure(figsize=(6, 4), dpi=80)
G = nx.DiGraph()

for record in records:
    G.add_edge(
        record.metadata["subject"],
        record.metadata["object_"],
        label=record.metadata["predicate"],
    )

pos = nx.spring_layout(G)
nx.draw(
    G,
    pos,
    with_labels=True,
    node_size=3000,
    node_color="lightblue",
    font_size=10,
    font_weight="bold",
    arrows=True,
)
edge_labels = nx.get_edge_attributes(G, "label")
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color="red")
plt.show()